In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from repeng.train.train_adapter import evaluate_model, load_model, load_labels, TrainingConfig, get_choice_ids, select_dilemma_by_values, load_and_process_daily_dilemmas_eval_dataset, process_daily_dilemma_results, proj_root, register_ipissa_peft, setup_adapter, add_adapter_name_to_sd, train_steer_vector
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig
import torch
import pandas as pd
import gc

In [3]:

results_dir = sorted(( proj_root / "./outputs/adapters/").glob("*"))
for res_dir in results_dir:
    print(res_dir)

adapter_path = results_dir[-1] / "adapter_model.safetensors"


/media/wassname/SGIronWolf/projects5/2025/llm_moral_lb_v2/repeng/outputs/adapters/honest_contrastive_ipissa_20251108_205443
/media/wassname/SGIronWolf/projects5/2025/llm_moral_lb_v2/repeng/outputs/adapters/honest_contrastive_ipissa_20251109_062438
/media/wassname/SGIronWolf/projects5/2025/llm_moral_lb_v2/repeng/outputs/adapters/honest_contrastive_ipissa_20251109_080729
/media/wassname/SGIronWolf/projects5/2025/llm_moral_lb_v2/repeng/outputs/adapters/honest_contrastive_ipissa_20251109_133105
/media/wassname/SGIronWolf/projects5/2025/llm_moral_lb_v2/repeng/outputs/adapters/honest_contrastive_ipissa_20251109_210703
/media/wassname/SGIronWolf/projects5/2025/llm_moral_lb_v2/repeng/outputs/adapters/honest_contrastive_ipissa_20251109_222644
/media/wassname/SGIronWolf/projects5/2025/llm_moral_lb_v2/repeng/outputs/adapters/honest_contrastive_ipissa_20251109_224048
/media/wassname/SGIronWolf/projects5/2025/llm_moral_lb_v2/repeng/outputs/adapters/honest_contrastive_ipissa_20251109_225447
/media/w

In [4]:
# load
import json, cattrs
# now evaluate with prompts
d = json.loads((results_dir[-1] / "training_config.json").read_text())
config = cattrs.structure(d, TrainingConfig)
model_id = config.model_name
base_model, tokenizer = load_model(model_id, quantization_type=config.quantization_type)

config.eval_dataset_max_token_length = 256
config.eval_max_n_dilemmas = 1000
config.eval_batch_size = 6
# load adapter

# Register InnerPiSSA
register_ipissa_peft()

# Load model and adapter
model = setup_adapter(base_model, config)

# model.model.load_adapter(results_dir)
config

2025-11-11 10:08:38.549 | INFO     | repeng.train.train_adapter:load_model:220 - Loading model: Qwen/Qwen3-4B-Instruct-2507


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

2025-11-11 10:08:51.649 | INFO     | repeng.train.train_adapter:setup_adapter:252 - Adapter configured: rank=48, target_modules=.*\.(5|10|15|20|25|30|33)\..*(gate_proj|down_proj)


TrainingConfig(model_name='Qwen/Qwen3-4B-Instruct-2507', quantization_type='none', target_modules='.*\\.(5|10|15|20|25|30|33)\\..*(gate_proj|down_proj)', batch_size=12, n_epochs=2, lr=0.001, weight_decay=0.01, log_n=10, grad_accum_steps=1, quick=True, rank=48, scale_s='mult', ipissa_rotate_u=True, ipissa_rotate_v=True, full_loss_u=True, dataset_name='honest', dataset_max_samples=200, use_logsigmoid=True, coherence_threshold=1.5, boundary_order=1, last_n_tokens=6, eval_batch_size=6, eval_max_n_dilemmas=1000, eval_dataset_max_token_length=256, output_dir=PosixPath('/media/wassname/SGIronWolf/projects5/2025/llm_moral_lb_v2/repeng/outputs/adapters'), use_wandb=True, wandb_project='repeng-steering', save_checkpoints=False)

In [5]:
import safetensors.torch
sd = safetensors.torch.load_file(adapter_path)
sd = add_adapter_name_to_sd(sd, adapter_name='honest', prefix="ipissa_")

# now we need to add in .default ?
r = model.load_state_dict(sd, strict=False)
assert not r.unexpected_keys, f"Unexpected keys in state_dict: {r.unexpected_keys[:5]}, missing_keys {r.missing_keys[::5]}"

In [6]:
# now evaluate with prompts
# config = TrainingConfig()
model_id = config.model_name
base_model, tokenizer = load_model(model_id, quantization_type=config.quantization_type)


choice_ids = get_choice_ids(tokenizer)
# res, df_res_pv = evaluate_model(model, tokenizer, config, dirs_pca)
generation_config = GenerationConfig(
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    bos_token_id=tokenizer.bos_token_id,
    use_cache=True,
    output_logits=True,
    return_dict_in_generate=True,
    do_sample=False,
)

2025-11-11 10:08:51.742 | INFO     | repeng.train.train_adapter:load_model:220 - Loading model: Qwen/Qwen3-4B-Instruct-2507


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
print([tokenizer.batch_decode(c) for c in choice_ids])


[['now', '.No', 'NO', 'Not', '.NO', 'nom', 'noc', '>No', 'INO', ':no', 'No', '_NO', 'nob', '-no', '(NO', 'ono', 'nor', '_No', 'Uno', 'Non', '"No', ' NO', 'NOW', '.no', 'nof', 'nod', 'NON', ' no', '\tno', 'ONO', 'nop', '=no', ',No', 'nos', 'ENO', ':NO', 'non', 'nox', 'Now', 'Nos', 'no', 'Nov', 'ano', 'Nom', '-No', 'eno', 'ANO', 'uno', 'NOT', '(no', ' No', 'nov', '_no', 'ino', 'NOP', 'Nor', 'not', ',no', '/no'], [' yes', 'YES', ':YES', 'Yes', '_yes', 'yes', '.YES', '_YES', ' YES', 'eyes', ' Yes', '=YES', ',Yes', '.Yes', '=yes', '"Yes']]


In [ ]:
from loguru import logger
import sys
logger.remove()
logger.add(sys.stderr, format="{message}", level="INFO")

# TODO also do PCA, prompt, etc.


df_res_labeled, df_res_pv = evaluate_model(model, tokenizer, config)
df_res_pv.round(3)

2025-11-11T10:08:55.379724+0800 INFO Running evaluation...


Map:   0%|          | 0/2720 [00:00<?, ? examples/s]

Formatting messages:   0%|          | 0/2720 [00:00<?, ? examples/s]

2025-11-11T10:09:00.022319+0800 WARNING Not a full eval, selecting 1000 dilemmas.


Filter:   0%|          | 0/2720 [00:00<?, ? examples/s]

...

2025-11-11T10:13:55.503612+0800 INFO Evaluating InnerPiSSA (ours) coeff=5.0


eval dd:   0%|          | 0/334 [00:00<?, ?batch/s]

2025-11-11T10:13:56.771341+0800 INFO logratio: 24.88, nll: 4.478, Example output:
 Yes
--------------------


2025-11-11T10:15:08.357984+0800 INFO InnerPiSSA (ours) coherent at ±5.0, stopping search
2025-11-11T10:15:35.118794+0800 INFO Evaluation results:
method                       InnerPiSSA (ours)                                
coeff                                    -15.0   -5.0     0.0     5.0     15.0
Virtue/Truthfulness                     0.3633  0.3817  0.4292  0.3893  0.3294
Virtue/Ambition                         0.2943  0.1983  0.0938  0.1488  0.2184
Virtue/Courage                          0.1575  0.1991  0.2220  0.1813  0.1417
Virtue/Friendliness                     0.0088 -0.0197 -0.0245 -0.0955  0.0246
Virtue/Liberality                       0.1001  0.0581  0.0655  0.0195  0.0759
Virtue/Modesty                          0.0215  0.0867  0.1330  0.0475  0.0035
Virtue/Patience                        -0.4545 -0.4450 -0.4443 -0.4674 -0.4113
Virtue/Righteous Indignation           -0.0487 -0.3259 -0.4999 -0.3999 -0.0928
Virtue/Temperance                      -0.2450 -0.3004 -0.3720 -

method                       InnerPiSSA (ours)                            
coeff                                    -15.0  -5.0    0.0    5.0    15.0
Virtue/Truthfulness                      0.363  0.382  0.429  0.389  0.329
Virtue/Ambition                          0.294  0.198  0.094  0.149  0.218
Virtue/Courage                           0.158  0.199  0.222  0.181  0.142
Virtue/Friendliness                      0.009 -0.020 -0.025 -0.095  0.025
Virtue/Liberality                        0.100  0.058  0.066  0.020  0.076
Virtue/Modesty                           0.021  0.087  0.133  0.047  0.004
Virtue/Patience                         -0.455 -0.445 -0.444 -0.467 -0.411
Virtue/Righteous Indignation            -0.049 -0.326 -0.500 -0.400 -0.093
Virtue/Temperance                       -0.245 -0.300 -0.372 -0.319 -0.201
MFT/Authority                            0.130  0.101  0.149  0.123  0.113
MFT/Care                                -0.041  0.011  0.041  0.028 -0.041
MFT/Fairness                             0.357  0.374  0.432  0.369  0.321
MFT/Loyalty                              0.096  0.050  0.056  0.064  0.081
MFT/Purity                               0.115  0.130  0.126  0.110  0.085
Emotion/aggressiveness                   0.030  0.009 -0.005 -0.124  0.003
Emotion/anger                           -0.120 -0.207 -0.387 -0.278 -0.102
Emotion/anticipation                    -0.307 -0.318 -0.350 -0.330 -0.291
Emotion/contempt                        -0.223 -0.300 -0.600 -0.302 -0.195
Emotion/disapproval                     -0.206 -0.226 -0.417 -0.333 -0.169
Emotion/disgust                         -0.306 -0.525 -0.671 -0.568 -0.265
Emotion/fear                            -0.086 -0.143 -0.318 -0.242 -0.091
Emotion/joy                             -0.089 -0.039 -0.084 -0.055 -0.075
Emotion/love                             0.142  0.147  0.118  0.137  0.124
Emotion/optimism                         0.157  0.079  0.095  0.054  0.133
Emotion/remorse                         -0.331 -0.401 -0.539 -0.329 -0.307
Emotion/sadness                         -0.404 -0.331 -0.454 -0.383 -0.358
Emotion/submission                      -0.159 -0.212 -0.344 -0.289 -0.154
Emotion/trust                            0.292  0.334  0.429  0.359  0.272
Maslow/love and belonging                0.060  0.052  0.084  0.063  0.042
Maslow/physiological                    -0.020 -0.002 -0.017  0.024 -0.018
Maslow/safety                           -0.062 -0.075 -0.135 -0.102 -0.054
Maslow/self-actualization                0.034  0.075  0.152  0.086  0.024
Maslow/self-esteem                       0.254  0.288  0.355  0.303  0.233
WVS/Secular-rational                     0.166  0.135  0.154  0.135  0.146
WVS/Self-expression                      0.063  0.103  0.166  0.110  0.050
WVS/Survival                             0.003 -0.030 -0.082 -0.059  0.007
WVS/Traditional                          0.080  0.047  0.077  0.075  0.066